In [1]:
import os
import pandas
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
wic_path  = "/content/drive/My Drive/datasets/WiC_dataset"


In [4]:
! ls "/content/drive/My Drive/datasets/WiC_dataset"

dev  README.txt  test  train


In [5]:
wic_train_path_d = os.path.join(wic_path, "train/train.data.txt")
wic_train_path_g = os.path.join(wic_path, "train/train.gold.txt")

train_data_df = pandas.read_csv(wic_train_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
train_gold_df = pandas.read_csv(wic_train_path_g, sep = "\t", names = ['label'])

train_data_df[['position1', 'position2']] = train_data_df['position1'].str.split("-", expand = True)
train_data_df = train_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

train_data_df.head()
# train_gold_df.head()

,word,pos,position1,position2,sent1,sent2
0,carry,V,2,1,You must carry your camping gear .,Sound carries well over water .
1,go,V,2,6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?
2,break,V,0,2,Break an alibi .,The wholesaler broke the container loads into ...
3,cup,N,8,4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .
4,academy,N,1,2,The Academy of Music .,The French Academy .


In [6]:
# do same for dev data
wic_dev_path_d = os.path.join(wic_path, "dev/dev.data.txt")
wic_dev_path_g = os.path.join(wic_path, "dev/dev.gold.txt")

dev_data_df = pandas.read_csv(wic_dev_path_d, sep = "\t", names = ['word', 'pos', 'position1', 'sent1', 'sent2'])
dev_gold_df = pandas.read_csv(wic_dev_path_g, sep = "\t", names = ['label'])

dev_data_df[['position1', 'position2']] = dev_data_df['position1'].str.split("-", expand = True)
dev_data_df = dev_data_df[['word', 'pos', 'position1', 'position2', 'sent1', 'sent2']]

dev_data_df.head()
# dev_gold_df.head()

,word,pos,position1,position2,sent1,sent2
0,board,N,2,2,Room and board .,He nailed boards across the windows .
1,circulate,V,0,4,Circulate a rumor .,This letter is being circulated among the facu...
2,hook,V,0,1,Hook a fish .,"He hooked a snake accidentally , and was so sc..."
3,recreation,N,1,9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...
4,domesticity,N,4,6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...


In [7]:
print((dev_data_df['sent1']))

0                                       Room and board .
1                                    Circulate a rumor .
2                                          Hook a fish .
3      For recreation he wrote poetry and solved cros...
4                        Making a hobby of domesticity .
                             ...                        
633    He told the painter that he wanted a yellow ba...
634    He needed a lot of power to hit the ball out o...
635           ` brunch ' is a well - known portmanteau .
636                       I promise you my best effort .
637                                To pierce a mystery .
Name: sent1, Length: 638, dtype: object


In [8]:
!nvidia-smi

Tue Oct 13 18:59:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [10]:
!pip install transformers

     |████████████████████████████████| 1.1MB 22.1MB/s 
     |████████████████████████████████| 3.0MB 56.0MB/s 
     |████████████████████████████████| 1.1MB 50.6MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=60c93e00992b2294d08750d37a5d7925b47dec13f7694891ce5884a729b0dc4a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [12]:
# sentences is a list 0f str
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_data_df['sent1'])
list_train_sentences_2 = list(train_data_df['sent2'])

list_dev_sentences_1 =  list(dev_data_df['sent1'])
list_dev_sentences_2 = list(dev_data_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i].split()
  # print(sentence_1)
  p1 = int(train_data_df['position1'][i])
  sentence_1 = sentence_1[:p1]+['"'+sentence_1[p1]+'"']+ sentence_1[p1+1:]
  sentence_1 = " ".join(sentence_1)
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i].split()
  # print(sentence_2)
  p2 = int(train_data_df['position2'][i])
  sentence_2 = sentence_2[:p2]+['"'+sentence_2[p2]+'"']+ sentence_2[p2+1:]
  sentence_2 = " ".join(sentence_2)
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i].split()
  p1 = int(dev_data_df['position1'][i])
  d_sentence_1 = d_sentence_1[:p1]+['"'+d_sentence_1[p1]+'"']+ d_sentence_1[p1+1:]
  d_sentence_1 = " ".join(d_sentence_1)
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i].split()
  p2 = int(dev_data_df['position2'][i])
  d_sentence_2 = d_sentence_2[:p2]+['"'+d_sentence_2[p2]+'"']+ d_sentence_2[p2+1:]
  d_sentence_2 = " ".join(d_sentence_2)
  dev_sentences_2.append(d_sentence_2)

  # print((dev_sentences_1[i], dev_sentences_2[i]))

print(len(dev_sentences_1))

638


In [13]:
encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_1, padding = True, truncation = True, return_tensors = 'pt')
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt')
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

79
<class 'torch.Tensor'>
64
<class 'torch.Tensor'>


In [14]:
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_gold_df.replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
print(train_labels)

dev_gold_df_tmp = dev_gold_df.replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)
print(len(dev_labels))

tensor([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]])
638


In [15]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_labels)


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

**CUTOFF**

In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, input_ids, token_type_ids, attention_mask, labels):
        loss, logits = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels=labels)[:2]

        return loss, logits

In [18]:
model = BERTi().to(device)
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The BERT model has 201 different named parameters.

==== Embedding Layer ====

encoder.bert.embeddings.word_embeddings.weight          (30522, 768)
encoder.bert.embeddings.position_embeddings.weight        (512, 768)
encoder.bert.embeddings.token_type_embeddings.weight        (2, 768)
encoder.bert.embeddings.LayerNorm.weight                      (768,)
encoder.bert.embeddings.LayerNorm.bias                        (768,)

==== First Transformer ====

encoder.bert.encoder.layer.0.attention.self.query.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.self.query.bias        (768,)
encoder.bert.encoder.layer.0.attention.self.key.weight    (768, 768)
encoder.bert.encoder.layer.0.attention.self.key.bias          (768,)
encoder.bert.encoder.layer.0.attention.self.value.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.self.value.bias        (768,)
encoder.bert.encoder.layer.0.attention.output.dense.weight   (768, 768)
encoder.bert.encoder.layer.0.attention.output.dense.bias  

In [19]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

# scheduler??


In [20]:
def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss()(output, targets)

In [21]:
# from transformers import get_linear_schedule_with_warmup

# # Number of training epochs. The BERT authors recommend between 2 and 4. 
# # We chose to run for 4, but we'll see later that this may be over-fitting the
# # training data.
# epochs = 10

# # Total number of training steps is [number of batches] x [number of epochs]. 
# # (Note that this is not the same as the number of training samples).
# total_steps = len(train_dataloader) * epochs

# # Create the learning rate scheduler.
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps = 0, # Default value in run_glue.py
#                                             num_training_steps = total_steps)

In [22]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [25]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 60 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy(all_dev_logits, all_dev_labels)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    # Do same on training data
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in train_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            loss, logits = model(b_input_ids, b_token_type_ids, b_attention_mask, b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy(all_dev_logits, all_dev_labels)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    print("  Training Validation took: {:}".format(validation_time))


print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 10 ========
Training...
  Batch    60  of    340.    Elapsed: 0:00:18.
  Batch   120  of    340.    Elapsed: 0:00:36.
  Batch   180  of    340.    Elapsed: 0:00:55.
  Batch   240  of    340.    Elapsed: 0:01:13.
  Batch   300  of    340.    Elapsed: 0:01:31.

  Average training loss: 0.69
  Training epcoh took: 0:01:43

Running Validation...
  Accuracy: 0.5549
  Validation Loss: 0.685
  Validation took: 0:00:03
  Accuracy: 0.6700
  Train Avg Loss: 0.639
  Training Validation took: 0:00:30

======== Epoch 2 / 10 ========
Training...
  Batch    60  of    340.    Elapsed: 0:00:18.
  Batch   120  of    340.    Elapsed: 0:00:36.
  Batch   180  of    340.    Elapsed: 0:00:55.
  Batch   240  of    340.    Elapsed: 0:01:13.
  Batch   300  of    340.    Elapsed: 0:01:31.

  Average training loss: 0.63
  Training epcoh took: 0:01:43

Running Validation...
  Accuracy: 0.5502
  Validation Loss: 0.700
  Validation took: 0:00:03
  Accuracy: 0.7644
  Train Avg Loss: 0.523
  Traini